# <p style="text-align: center;"> Задание SQL </p>

**Цель исследования:** провести анализ данных сервиса книг и решить следующие задачи:<br>

+ сколько книг вышло после 1 января 2000 года;
+ посчитать количество обзоров и среднюю оценку каждой книги;
+ определить издательство, которое выпустило наибольшее число книг толще 50 страниц — чтобы исключитеь из анализа брошюры;
+ определить автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками;
+ посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

# Описание данных

**Таблица `books`**<br>
Содержит данные о книгах:<br>
`book_id` — идентификатор книги;<br>
`author_id` — идентификатор автора;<br>
`title` — название книги;<br>
`num_pages` — количество страниц;<br>
`publication_date` — дата публикации книги;<br>
`publisher_id` — идентификатор издателя.

**Таблица `authors`**<br>
Содержит данные об авторах:<br>
`author_id` — идентификатор автора;<br>
`author` — имя автора.

**Таблица `publishers`**<br>
Содержит данные об издательствах:<br>
`publisher_id` — идентификатор издательства;<br>
`publisher` — название издательства;

**Таблица `ratings`**<br>
Содержит данные о пользовательских оценках книг:<br>
`rating_id` — идентификатор оценки;<br>
`book_id` — идентификатор книги;<br>
`username` — имя пользователя, оставившего оценку;<br>
`rating` — оценка книги.

**Таблица `reviews`**<br>
Содержит данные о пользовательских обзорах на книги:<br>
`review_id` — идентификатор обзора;<br>
`book_id` — идентификатор книги;<br>
`username` — имя пользователя, написавшего обзор;<br>
`text` — текст обзора.

In [1]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import inspect

In [2]:
# устанавливаем параметры
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

# Изучение информации о таблицах

In [3]:
def data_head(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Функция для вывода первых 5 строк таблицы'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

In [4]:
def data_count(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Функция для вывода подсчета количества строк таблицы'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

In [5]:
insp = inspect(engine)
display(pd.DataFrame(insp.get_columns('books'))[['name','type']])

,name,type
0,book_id,INTEGER
1,author_id,INTEGER
2,title,TEXT
3,num_pages,INTEGER
4,publication_date,DATE
5,publisher_id,INTEGER


In [6]:
data_head('''SELECT * FROM books LIMIT 5''')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Таблица `books` содержит 1000 строк и следующие столбцы: `book_id`,`author_id`,`title`,`num_pages`,`publication_date`,`publisher_id`.**

In [7]:
display(pd.DataFrame(insp.get_columns('authors'))[['name','type']])

,name,type
0,author_id,INTEGER
1,author,TEXT


In [8]:
data_head('''SELECT * FROM authors LIMIT 5''')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [9]:
data_count('''SELECT COUNT (*) FROM authors''')

,count
0,636


**Таблица `authors` содержит 636 строк и следующие столбцы: `author_id`, `author`.**

In [10]:
display(pd.DataFrame(insp.get_columns('publishers'))[['name','type']])

,name,type
0,publisher_id,INTEGER
1,publisher,TEXT


In [11]:
data_head('''SELECT * FROM publishers LIMIT 5''')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [12]:
data_count('''SELECT COUNT (*) FROM publishers''')

,count
0,340


In [13]:
display(pd.DataFrame(insp.get_columns('ratings'))[['name','type']])

,name,type
0,rating_id,INTEGER
1,book_id,INTEGER
2,username,TEXT
3,rating,INTEGER


**Таблица `publishers` содержит 340 строк и следующие столбцы: `publisher_id`, `publisher`.**

In [14]:
data_head('''SELECT * FROM ratings LIMIT 5''')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [15]:
data_count('''SELECT COUNT (*) FROM ratings''')

,count
0,6456


**Таблица `ratings` содержит 6456 строк и следующие столбцы: `rating_id`, `book_id`, `username`, `rating`.**

In [16]:
display(pd.DataFrame(insp.get_columns('reviews'))[['name','type']])

,name,type
0,review_id,INTEGER
1,book_id,INTEGER
2,username,TEXT
3,text,TEXT


In [17]:
data_head('''SELECT * FROM reviews LIMIT 5''')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [18]:
data_head('''SELECT COUNT (*) FROM reviews''')

,count
0,2793


**Таблица `reviews` содержит 2793 строки и следующие столбцы: `review_id`, `book_id`, `username`, `text`.**

* сколько книг вышло после 1 января 2000 года;
* посчитать количество обзоров и среднюю оценку каждой книги;
* определить издательство, которое выпустило наибольшее число книг толще 50 страниц — чтобы исключитеь из анализа брошюры;
* рпределить автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками;
* посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

+ Сделайте по одному SQL-запросу для решения каждого задания;
+ Выведите результат каждого запроса в тетрадке;
+ Опишите выводы по каждой из решённых задач.

# Задача 1. Сколько книг вышло после 1 января 2000 года

In [19]:
books_2000 = '''SELECT COUNT(*)
                FROM books
                WHERE publication_date >= '2000-01-01'
                '''

data_books_2000 = pd.io.sql.read_sql(books_2000, con = engine)
print('После 01 января 2020 года вышла {} книга '.format(data_books_2000.loc[0, 'count']))

После 01 января 2020 года вышла 821 книга 


<div class="alert alert-warning">
<b>Комментарий ревьюера v.1</b>
    
📝 Если подходить строго к этому запросу, то в нем мы должны были вывести значения после 1 января 2000, т.е. не включая эту дату, поэтому оператор сравнения должен быть `>`.

</div>

# Задача 2. Для каждой книги посчитайте количество обзоров и среднюю оценку

In [20]:
rating = '''SELECT b.title,
            COUNT (DISTINCT rw.review_id), ROUND (avg(r.rating), 2) as R
            FROM REVIEWS AS rw
            RIGHT OUTER JOIN BOOKS AS b ON rw.book_id=b.book_id
            LEFT OUTER JOIN RATINGS AS r ON b.book_id = r.book_id
            GROUP BY b.book_id
            ORDER BY R DESC'''

book_rating = pd.io.sql.read_sql(rating, con = engine)
book_rating.columns = ['Название', 'Количество обзоров', 'Средний рейтинг']
book_rating

,Название,Количество обзоров,Средний рейтинг
0,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,Light in August,2,5.00
3,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,His Excellency: George Washington,2,2.00
997,Drowning Ruth,3,2.00
998,Junky,2,2.00


# Задача 3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц

In [21]:
books_max = '''SELECT p.publisher,
               COUNT(b.book_id)
               FROM publishers AS p
               JOIN BOOKS AS b ON p.publisher_id=b.publisher_id
               WHERE b.num_pages > 50
               GROUP BY p.publisher_id
               ORDER BY count(b.book_id) DESC
               LIMIT 1'''

max_books = pd.io.sql.read_sql(books_max, con = engine)
max_books.columns = ['Название издательства', 'Количество книг']
max_books

,Название издательства,Количество книг
0,Penguin Books,42


# Задача 4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [24]:
autor_max = '''SELECT a.author,
               AVG(r.rating) avg_rating
               FROM authors AS a
               JOIN books AS b ON a.author_id=b.author_id
               JOIN ratings AS r ON b.book_id=r.book_id
               WHERE b.book_id IN
                    (SELECT b.book_id
                     FROM books AS b
                     JOIN ratings AS r ON b.book_id=r.book_id
                     GROUP BY b.book_id
                     HAVING count(r.rating_id)>=50)
               GROUP BY a.author
               ORDER BY avg_rating desc
               LIMIT 1'''

max_autor = pd.io.sql.read_sql(autor_max, con = engine)
max_autor.columns = ['Автор', 'Средний рейтинг']
max_autor
print('Автор с самой высокой средней оценкой книг это {} '.format('\033[1m' + max_autor.loc[0, 'Автор']) )

Автор с самой высокой средней оценкой книг это J.K. Rowling/Mary GrandPré 


,Автор,Средний рейтинг
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Lois Lowry,3.750000


In [29]:
# ячейка ревьюера:

autor_max = '''
-- В начале найдем книги и средний рейтинг для каждой из них
WITH book_r AS (
    SELECT 
        b.author_id,
        b.book_id,
        AVG(r.rating) avg_rating
    FROM books AS b
    JOIN ratings AS r ON b.book_id=r.book_id
    GROUP BY b.book_id
    HAVING count(r.rating_id)>=50
)

-- Затем считаем средний рейтинг для каждого автора по среднему рейтингу книг
SELECT 
    a.author,
    AVG(br.avg_rating) avg_rating,
    COUNT(br.avg_rating) books_count -- и выведем количество книг, по которым прошел подсчет среднего
FROM authors AS a
JOIN book_r AS br ON a.author_id=br.author_id

GROUP BY a.author
ORDER BY avg_rating desc

'''

max_autor = pd.io.sql.read_sql(autor_max, con = engine)
max_autor

,author,avg_rating,books_count
0,J.K. Rowling/Mary GrandPré,4.283844,4
1,Markus Zusak/Cao Xuân Việt Khương,4.264151,1
2,J.R.R. Tolkien,4.258446,2
3,Louisa May Alcott,4.192308,1
4,Rick Riordan,4.080645,1
5,William Golding,3.901408,1
6,J.D. Salinger,3.825581,1
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474,1
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879,1
9,Dan Brown,3.754540,2


# Задача 5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [23]:
mean_users = '''WITH A AS
                    (SELECT username,
                     COUNT(review_id)
                     FROM reviews
                     WHERE username in
                         (SELECT username
                          FROM ratings
                          GROUP BY username
                          HAVING count(rating_id)>48)
                    GROUP BY username)
                SELECT round (SUM (COUNT) / COUNT(COUNT), 2) AS AVG
                FROM A'''

users_mean = pd.io.sql.read_sql(mean_users, con = engine)

print ('Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет ', users_mean.loc[0, 'avg'])

Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет  24.0


**Вывод:**<br>
1. После 01 января 2020 года вышла 821 книга;
2. Посчитано количество обзоров и средняя оценка для каждой книги;
3. Издательство `Penguin Books`	выпустило 42 книги, это максимальное количество;
4. Автор с самой высокой средней оценкой книг это J.K. Rowling/Mary GrandPré;
5. Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет  24.0